In [1]:
# loading in html parser library
from bs4 import BeautifulSoup
import pprint as pp
import inflect
p = inflect.engine()

# loading in html file from Canadian Mammal knowledge base website
f = open('./all.html', 'r')
cmFile = BeautifulSoup(f.read(), 'html.parser')


In [2]:
# option to print html file in nice format 
# print(cmFile.prettify())


# cmFile.find_all('a')[1]
# print(len(cmFile.table))
# data = cmFile.p.table
# moreData = data.find_all('tr')

# for index, item in enumerate(cmFile.find_all('table')[1].find_all('tr')):
# for index, item in enumerate(cmFile.find_all('table')[1].tr):

# setting up list of dictionaries for each animal
animalList = []
animal = {'name': [], 'prop': [], 'relation': []}

# parsing html to get properties of each animal 
    # in this html file, animals have three <tr> sections each
    # in the first <tr> section, there are three <td> sections, the first being the name of the animal
    # in the second and third <tr> sections, there are only two <td> sections
    # the last <td> section, regardless of <tr> section, does not list a property of the animal 

for item in cmFile.p.table.children:
    if hasattr(item, 'children'):
        # if number of <td> in <tr> == 3
        if len(list(item.children)) == 3:
            # appending previous animal dict to list of animals 
            if len(animal['prop']) > 0:
                animalList.append(animal)
            # resetting animal dictionary template
            animal = dict([('name',[]), ('prop',[]), ('relation',[])])

            for idx, val in enumerate(item.children):
                # if first <td>, save value as name
                if idx == 0:
                    animal['name'].append(val.get_text().strip())
                # if second <td>, save values as property and relations
                if idx == 1:
                    animal['prop'].append(val.get_text().strip().split(","))
                    animal['relation'].append(val.b.get_text().strip())
        else:
            for idx, val in enumerate(item.children):
                # if second <td>, save values as property and relations
                if idx == 0:
                    animal['prop'].append(val.get_text().strip().split(","))
                    animal['relation'].append(val.b.get_text().strip())                  

In [761]:
animalList_d = []

# saving triplets
for index, animal in enumerate((animalList)):
    for idx, value in enumerate(animalList[index]['prop']):
        for indx, prop in enumerate(animalList[index]['prop'][idx]):
            animalList_d.append([animalList[index]['name'][0], animalList[index]['relation'][idx], animalList[index]['prop'][idx][indx]])

tempStore1 = []
tempStore2 = []

# getting birth months for animals 
for line in animalList_d:
    if line[1].startswith("gives birth in"):
        tempStore1 = line[2].split()
        for word in tempStore1:
            if not word.islower():
                tempStore2.append(word)
        
        if len(tempStore2) == 1:
            line[2] = tempStore2[0]
            tempStore2 = []
        
        if len(tempStore2) > 1:
            for index, value in enumerate(tempStore2):
                animalList_d.append([line[0], line[1], tempStore2[index].replace("mid-","")])
            tempStore2 = []

animalList_d = [line for line in animalList_d if not line[2].startswith("gives birth in")]
           

# deleting relation text from property text                                  
for line in animalList_d:
    if line[2].startswith("can"):
        line[2] = line[2].replace("can","").strip()
        line[1] = ''.join(word if word == 'can' else '' for word in line[1].split()).strip()
    
    if line[1].startswith("has good"):
        line[2] = line[2].replace("has","")
        line[1] = line[1].replace(line[2],"")
        
    if line[1].startswith("has poor"):
        line[2] = line[2].replace("has","")
        line[1] = line[1].replace(line[2],"")
    
    if not line[2].startswith("is a kind of"):
        if line[2].startswith("is"): 
            line[2] = line[2].replace("is","").strip()
            line[1] = ''.join(word if word == 'is' else '' for word in line[1].split()).strip()
            
        else: 
            line[2] = line[2].replace(line[1],"").lstrip().rstrip(' ')
    
    else: 
            line[2] = line[2].replace(line[1],"").lstrip().rstrip(' ')
    
    line[2] = line[2].lstrip()
    
# TODO fix gestation period
# TODO average number of babies
# TODO order, family, and scientific name
# delete french name (or other language names)


# removing non-binary properties    
animalList_d = [line for line in animalList_d if not line[1] == "has picture"]
animalList_d = [line for line in animalList_d if not line[1] == "has range"]
animalList_d = [line for line in animalList_d if not line[1] == "has distribution"]
animalList_d = [line for line in animalList_d if not line[1] == "has population density"]
animalList_d = [line for line in animalList_d if not line[1] == "has height"]
animalList_d = [line for line in animalList_d if not line[1] == "has weight"]
animalList_d = [line for line in animalList_d if not line[1] == "has length"]
animalList_d = [line for line in animalList_d if not line[1] == "weighs"]

#pluralifying food 
# for line in animalList_d:
#     if line[1].startswith("eats"):
#         line[2] = p.singular_noun(line[2])


In [764]:
import csv
 
def csv_writer(data, path):
    """
    Write data to a CSV file path
    """
    with open(path, "w", newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(data)

# def csv_writer(data, path):
#     with open('names.csv', 'w', newline='') as csvfile:
#         fieldnames = ['name', 'prop', 'relation']
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#         for row in data:
#             writer.writerow(data[row]{'name', 'prop', 'relation'})


# def csv_writer(data, path, keys):
#     with open(path, 'w') as output_file:
#         dict_writer = csv.DictWriter(output_file, keys)
#         dict_writer.writer.writerow(keys)
#         dict_writer.writerows(data)

# for exporting dictionaries
# def csv_writer(data, path, keys):
#     with open('out1.csv','w',newline='') as f:
#         w = csv.writer(f)
#         w.writerow(['list_id','list_items'])
#         for animal in data: 
#             for key,items in animal.items():
#                 for item in items:
#                     w.writerow([key,item])
        
path = "output1.csv"
# animalKeys = animalList[0].keys()

csv_writer(animalList_d, path)



In [3]:
# isinstance(animalList_d,dict)
# a = []

# b = animalList_d[220][2].split()
# for word in b:    
#     if not word.islower():
#         a.append(word)

# print(a)
# print((animalList_d[220][2]))
# pp.pprint(animalList_d, width=200)
# # print(animalList_d[130])
# # print(animalList_d[42][1])

pp.pprint(animalList)

[{'name': ['adult Arctic fox'],
  'prop': [['eats lemmings',
            ' birds',
            ' eggs',
            ' ground squirrels',
            ' berries',
            ' small marine animals',
            ' fish',
            ' and carrion'],
           ['has colour white in winter', ' brown in summer'],
           ['has length 75-115 cm'],
           ['has picture'],
           ['is a kind of Arctic fox'],
           ['weighs 2.5 to 9kg']],
  'relation': ['eats',
               'has colour',
               'has length',
               'has picture',
               'is a kind of',
               'weighs']},
 {'name': ['adult badger'],
  'prop': [['digs with its front claws'],
           ['eats ground squirrels',
            ' pocket gophers',
            ' prairie dogs',
            ' kangaroo rats',
            ' pocket mouse',
            ' voles',
            ' birds',
            ' snails',
            ' insects'],
           ['fights courageously'],
           ['has breeding 

               'may adopt']},
 {'name': ['adult female wolverine'],
  'prop': [['attacks intruders to her nest when her babies are young'],
           ['gives birth in March and April'],
           ['has one breeding period per year'],
           ['has average number of babies 2-5'],
           ['has gestation period 7-8 months'],
           ['has length 865-932 mm'],
           ['is a kind of adult wolverine'],
           ['is a kind of female mammal'],
           ["licks her babies' tummies after feeding to aid digestion"],
           ['weighs 6.6-14.8 kg'],
           ['can breed when she is 2 years old']],
  'relation': ['attacks',
               'gives birth in',
               'has',
               'has average number of babies',
               'has gestation period',
               'has length',
               'is a kind of',
               'is a kind of',
               'licks',
               'weighs',
               'can breed']},
 {'name': ['adult fisher'],
  'prop': [['eats

  'relation': ['has length', 'is a kind of', 'is a kind of', 'weighs']},
 {'name': ['adult male lynx'],
  'prop': [['has length 74-106.5 cm'],
           ['is a kind of adult lynx'],
           ['is a kind of male mammal'],
           ['weighs 6.7-17.2 kg']],
  'relation': ['has length', 'is a kind of', 'is a kind of', 'weighs']},
 {'name': ['adult male marten'],
  'prop': [['has length 490-602 mm'],
           ['has range 2.5 square km'],
           ['is a kind of adult marten'],
           ['is a kind of male mammal'],
           ['is polygamous'],
           ['weighs 1000 g']],
  'relation': ['has length',
               'has range',
               'is a kind of',
               'is a kind of',
               'is polygamous',
               'weighs']},
 {'name': ['adult male mink'],
  'prop': [['has length 491-620 mm'],
           ['has range up to 7.8 square km'],
           ['is a kind of adult mink'],
           ['is a kind of male mammal'],
           ['reaches sexual maturity b

               'has picture',
               'has range',
               'has temperament',
               'is a kind of',
               'is good parent',
               'is most active at',
               'is omnivorous',
               'is usually monogamous',
               'males sound',
               'reaches sexual maturity by',
               'weighs']},
 {'name': ['adult river otter'],
  'prop': [['catches food under water'],
           ['eats fish',
            ' insects',
            ' frogs',
            ' tadpoles',
            ' newts',
            ' muskrats',
            ' meadow voles',
            ' shrews and beavers'],
           ['has breeding season late winter to early spring'],
           ['has colour brown'],
           ['has length 1-1.3 m'],
           ['has part long', ' streamlined body'],
           ['has part short legs'],
           ['has part small ears which may be closed under water'],
           ['has part thick', ' lustrous fur with dense underfur'

  'prop': [['has scientific name Lagenorhynchus acutus'],
           ['is a kind of dolphin or porpoise']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['baby badger'],
  'prop': [['is a kind of baby mammal'], ['is a kind of badger']],
  'relation': ['is a kind of', 'is a kind of']},
 {'name': ['baby black-footed ferret'],
  'prop': [['is a kind of baby mammal'],
           ['is a kind of black-footed ferret'],
           ['is playful']],
  'relation': ['is a kind of', 'is a kind of', 'is playful']},
 {'name': ['baby coyote'],
  'prop': [['is a kind of baby mammal'],
           ['is a kind of coyote'],
           ['is weaned by 1 month']],
  'relation': ['is a kind of', 'is a kind of', 'is weaned by']},
 {'name': ['baby ermine'],
  'prop': [['grows rapidly'],
           ['has colour white at birth'],
           ['has part sealed ears at birth'],
           ['has part sealed eyes at birth'],
           ['is a kind of baby mammal'],
           ['is a kind of ermine'

               'is eaten by',
               'slaps',
               'swims',
               'uses',
               'weighs']},
 {'name': ["Bendire's shrew"],
  'prop': [['has scientific name Sorex bendirii'], ['is a kind of shrew']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['big brown bat'],
  'prop': [['has scientific name Eptesicus fuscus'],
           ['is a kind of smooth-faced bat']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['big free-tailed bat'],
  'prop': [['has scientific name Tadarida sapiens'],
           ['is a kind of free-tailed bat']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['bighorn sheep'],
  'prop': [['has picture'],
           ['has scientific name Ovis canadensis'],
           ['is a kind of antelope', ' cattle', ' sheep or goat']],
  'relation': ['has picture', 'has scientific name', 'is a kind of']},
 {'name': ['bison'],
  'prop': [['eats grasses', ' sedges', ' forbs', ' and other grou

  'relation': ['eats',
               'has length',
               'has part',
               'has part',
               'is a kind of',
               'is a kind of',
               'is playful',
               'is weaned by',
               'loses spots at',
               'opens eyes at',
               'stays with',
               'weighs',
               'may be born in']},
 {'name': ['coyote'],
  'prop': [['has scientific name Canis latrans'],
           ['is a kind of dog'],
           ['is also called brush wolf'],
           ['is eaten by wolf', ' black bear', ' cougar', ' eagle'],
           ['makes sound yelping and howling cry',
            ' bark',
            ' growl',
            ' wail',
            ' and squeal']],
  'relation': ['has scientific name',
               'is a kind of',
               'is also called',
               'is eaten by',
               'makes sound']},
 {'name': ['coypu'],
  'prop': [['has scientific name Myocastor coypus'], ['is a kind of roden

           ['is a kind of rat', ' mouse or vole']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['long-tailed weasel'],
  'prop': [['has distribution southern Canada'],
           ['has habitat open grassland',
            ' aspen parklands',
            ' river bottom lands'],
           ['has name in French belette à longue queue'],
           ['has scientific name Mustela frenata'],
           ['is a kind of weasel']],
  'relation': ['has distribution',
               'has habitat',
               'has name in French',
               'has scientific name',
               'is a kind of']},
 {'name': ['lynx'],
  'prop': [['has habitat dense boreal forest', ' tundra'],
           ['has name in French loup-cervier'],
           ['has scientific name Lynx lynx'],
           ['is a kind of cat'],
           ['is killed by humans', ' cougars', ' wolves']],
  'relation': ['has habitat',
               'has name in French',
               'has scientific name',
        

               'is a kind of']},
 {'name': ["Ord's kangaroo rat"],
  'prop': [['has scientific name Dipdomys ordii'],
           ['is a kind of pocket mouse or kangaroo rat']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['Pacific coast mole'],
  'prop': [['has scientific name Scapanus orarius'], ['is a kind of mole']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['Pacific jumping mouse'],
  'prop': [['has scientific name Zapus trinotatus'],
           ['is a kind of jumping mouse or jerboa']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['Pacific pilot whale'],
  'prop': [['has scientific name Globicephala macrorhyncha'],
           ['is a kind of dolphin or porpoise']],
  'relation': ['has scientific name', 'is a kind of']},
 {'name': ['Pacific white-sided dolphin'],
  'prop': [['has scientific name Lagenorhynchus obliquidens'],
           ['is a kind of dolphin or porpoise']],
  'relation': ['has scientific name', 'is

            ' herbs',
            ' grass',
            ' fiddleheads',
            ' mushrooms',
            ' blueberries',
            ' acorns',
            ' twigs',
            ' buds'],
           ['has colour summer: red on the back and sides and white below',
            ' winter: grey above'],
           ['has height 1 m at the shoulder'],
           ['has population density more than three deer per square kilometer'],
           ['has range as far north as Great Slave Lake and from Cape Breton '
            'Island westward to south-central British Columbia'],
           ['has scientific name Odocoileus virginianus'],
           ['has subspecies northern white-tailed deer',
            ' Dakota white-tailed deer',
            ' tawny northwestern white-tailed deer'],
           ['is a kind of deer'],
           ['can weigh more than 110 kg']],
  'relation': ['eats',
               'has colour',
               'has height',
               'has population density',
           

In [ ]:
# deleting relation text from property text                                  
# for index, animal in enumerate((animalList)):
#     for idx, value in enumerate(animalList[index]['prop']):
#         if animalList[index]['prop'][idx].startswith("can"):
#             animalList[index]['prop'][idx] = animalList[index]['prop'][idx].replace("can","")
#             animalList[index]['relation'][idx] = animalList[index]['relation'][idx].replace(animalList[index]['prop'][idx],"")         
        
#         if not animalList[index]['prop'][idx].startswith("is a kind of"):
#             if animalList[index]['prop'][idx].startswith("is"): 
#                 animalList[index]['prop'][idx] = animalList[index]['prop'][idx].replace("is","")
#                 animalList[index]['relation'][idx] = animalList[index]['relation'][idx].replace(animalList[index]['prop'][idx],"")         

#             else: 
#                 animalList[index]['prop'][idx] = animalList[index]['prop'][idx].replace(animalList[index]['relation'][idx],"")
